$$\Huge\textbf{Image Processing}$$

# Imports

In [ ]:
# standard library imports
import os  #  directory and file operations
import shutil  #  copying files
import  time  #  adding delays

# installed library imports
from sklearn.model_selection import train_test_split  #  splitting datasets
from PIL import Image  #  image processing
import torchvision.transforms as transforms  #  data augmentation
import numpy as np  #  numerical operations

# Global Constants

In [ ]:
# Constants
PROJECT_DATASET_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_dataset'
TRAIN_VAL_DIR = os.path.join(PROJECT_DATASET_DIR, 'train_val')
TEST_DIR = os.path.join(PROJECT_DATASET_DIR, 'test')

TRAIN_DIR = os.path.join(PROJECT_DATASET_DIR, 'train')  # new directory
VALIDATION_DIR = os.path.join(PROJECT_DATASET_DIR, 'validation')  # new directory

IMAGE_SIZE = (256, 256)
VALIDATION_SPLIT = 15 / 85  # train_val is 85% of the total dataset, we want the validation set to be 15% of the total dataset
CLASSES = ['airplane_cabin', 'hockey_arena', 'movie_theater', 'staircase', 'supermarket']
RANDOM_SEED = 42
BATCH_SIZE = 100

In [ ]:
# We aim to obtain the following structure:

# project_code\
# │
# ├── data\
# │   └── project_dataset\  
# │       ├── train_val\
# │       │   ├── airplane_cabin\
# │       │   ├── hockey_arena\
# │       │   ├── movie_theater\
# │       │   ├── staircase\
# │       │   └── supermarket\
# │       │
# │       ├── test\  ---> normalized data (750 images in total)
# │       │   ├── airplane_cabin\  ---> 150 images 
# │       │   ├── hockey_arena\  ---> 150 images 
# │       │   ├── movie_theater\  ---> 150 images
# │       │   ├── staircase\  ---> 150 images 
# │       │   └── supermarket\  ---> 150 images 
# │       │
# │       ├── training\  ---> normalized and augmented data (14,000 images in total)
# │       │   ├── airplane_cabin\  ---> 4 x 700 = 2800 images
# │       │   ├── hockey_arena\  ---> 2800 images
# │       │   ├── movie_theater\  ---> 2800 images
# │       │   ├── staircase\  ---> 2800 images
# │       │   └── supermarket\  ---> 2800 images
# │       │
# │       └── validation\  ---> normalized data (750 images in total)
# │           ├── airplane_cabin\  ---> 150 images
# │           ├── hockey_arena\  ---> 150 images
# │           ├── movie_theater\  ---> 150 images
# │           ├── staircase\  ---> 150 images
# │           └── supermarket\  ---> 150 images
# │
# └── notebooks\  # Python code for data processing, model training, etc.
#     ├── create_project_dataset.ipynb   
#     ├── image_processing.ipynb  
#     └── ...  # other notebooks

# Training and Validation Sets Separation

In [ ]:
# create a new directory for train set and a new directory for validation set
def create_directories():
    """
    Create directories for processed data.
    
    No inputs or outputs. This function creates train and validation directories for each class.
    """
    os.makedirs(TRAIN_DIR, exist_ok=True)
    os.makedirs(VALIDATION_DIR, exist_ok=True)
    for class_name in CLASSES:
        os.makedirs(os.path.join(TRAIN_DIR, class_name), exist_ok=True)
        os.makedirs(os.path.join(VALIDATION_DIR, class_name), exist_ok=True)


In [ ]:
# copy images from the train_val directory to the new train and val directories
def copy_images_in_batches(src_dir, dest_dir, file_list, batch_size):
    """
    Copy images from the source directory to the destination directory in batches.
    Include a short delay to prevent overloading the file system.

    Inputs:
    - src_dir: Source directory containing the original images.
    - dest_dir: Destination directory where images will be copied.
    - file_list: List of image filenames to be copied.
    - batch_size: Number of images to copy in each batch.

    No outputs. The function copies files and prints the status of each batch.
    """
    for i in range(0, len(file_list), batch_size):
        batch = file_list[i:i + batch_size]
        for file_name in batch:
            src_file = os.path.join(src_dir, file_name)
            dest_file = os.path.join(dest_dir, file_name)
            shutil.copyfile(src_file, dest_file)
            print(f"Copied {src_file} to {dest_file}")
        time.sleep(0.5)  # add a short delay to prevent overloading the file system


In [ ]:
# distribute the images in train and validation sets
def split_train_val():
    """
    Split the data in train_val directory into training and validation sets.

    No inputs or outputs. This function splits the images and moves them to their respective directories.
    """
    for class_name in CLASSES:
        class_dir = os.path.join(TRAIN_VAL_DIR, class_name)
        images = os.listdir(class_dir)
        train_images, val_images = train_test_split(images, test_size=VALIDATION_SPLIT, random_state=RANDOM_SEED)

        # copy training images in batches
        copy_images_in_batches(class_dir, os.path.join(TRAIN_DIR, class_name), train_images, BATCH_SIZE)

        # copy validation images in batches
        copy_images_in_batches(class_dir, os.path.join(VALIDATION_DIR, class_name), val_images, BATCH_SIZE)


# Training, Validation, and Testing Data Normalization

In [ ]:
# normalize both training and validation images
def normalize_image(image):
    """
    Resize (to 256x256) and normalize the image using PyTorch transforms.
    
    Inputs:
    - image: PIL Image object.
    
    Output:
    - Normalized image as a PyTorch tensor.
    """
    transform = transforms.Compose([
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),  # converts image to PyTorch tensor and scales pixel values to [0, 1]
    ])
    return transform(image)

In [ ]:
def normalize_images_in_directory(directory):
    """
    Normalize all images in the specified directory.
    Normalized images are converted back to PIL images and overwrite the original ones.
    
    Inputs:
    - directory: Path to the directory containing images.
    
    No outputs. This function normalizes images and saves them back as PIL images.
    """
    for class_name in CLASSES:
        class_dir = os.path.join(directory, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            image = Image.open(img_path)
            normalized_image = normalize_image(image)
            normalized_image_pil = transforms.ToPILImage()(normalized_image)
            normalized_image_pil.save(img_path)  # overwrite the original image


# Training Data Augmentation

In [ ]:
# only augment training images
def augment_image(image):
    """
    Apply combined augmentation techniques (rotation, flipping, brightness enhancement) to the image using PyTorch transforms.
    Each original image generates three additional augmented images, increasing the dataset size.
    
    Inputs:
    - image: PIL Image object.
    
    Outputs:
    - List of augmented images as PIL Image objects.
    """
    # define the transforms
    transform = transforms.Compose([
        transforms.RandomRotation(15),  # randomly rotate the image by up to 15 degrees
        transforms.RandomHorizontalFlip(),  # randomly flip the image horizontally with a probability of 0.5
        transforms.ColorJitter(brightness=1.5)  # randomly change the brightness of the image
    ])
    
    # apply the transformations directly to the PIL image
    augmented_images = [transform(image) for _ in range(3)]  # give 3 augmentations
    return augmented_images



In [ ]:
def augment_images_in_directory(directory):
    """
    Apply augmentation to all images in the specified (training) directory.
    Augmented images are directly saved into the training directory.
    
    Inputs:
    - directory: Path to the directory containing images.
    
    No outputs. This function augments images and saves them in the same directory.
    """
    for class_name in CLASSES:
        class_dir = os.path.join(directory, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            image = Image.open(img_path)
            augmented_images = augment_image(image)
            for i, aug_image in enumerate(augmented_images):
                aug_image_path = os.path.join(class_dir, f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg")
                aug_image.save(aug_image_path)


# Code Execution

In [ ]:
# create training and validation directories
create_directories()
print("train and validation directories created sucessfully!")

In [ ]:
# distribute images into training and validation directories
split_train_val()
print("train and validation data split successfully!")

In [ ]:
# normalize training and validation images

normalize_images_in_directory(TRAIN_DIR)
print("training data normalized successfully!")

normalize_images_in_directory(VALIDATION_DIR)
print("validation data normalized successfully!")

In [ ]:
# normalize testing images
normalize_images_in_directory(TRAIN_DIR)
print("testing data normalized successfully!")

In [ ]:
# augment the training images
augment_images_in_directory(TRAIN_DIR)
print("training data augmented successfully!")